# <u> LSTM ENCODER DECODER

# Data

In [1]:
import utils
import numpy as np

#utils.setup_nltk()
PRESIDENT = 'obama'
speeches = utils.read_all_text_files(PRESIDENT)

In [2]:
assert np.mean([len(x) for x in speeches]) == 4878.9

### Preprocessing

In [3]:
filter_list = [':', '(', ')', ',', '-', '.']
filtered_speeches = []

for speech in speeches:
    filtered_speech = []
    for word in speech:
        # filter out unwanted words
        if word not in filter_list:
            # lower word
            filtered_speech.append(word.lower())
    filtered_speeches.append(filtered_speech)

### Create N-Grams

In [4]:
from nltk import ngrams

WINDOW = 3
grams = [ngrams(s, WINDOW+1) for s in filtered_speeches]
flat_grams = [ng for speech in grams for ng in speech]

### Split Data

In [5]:
import pandas as pd

X = [' '.join(list(x[0:WINDOW])) for x in flat_grams]
Y = [x[-1] for x in flat_grams]
df = pd.DataFrame.from_dict({'x':X, 'y':Y})

# persist
csv_name = '../data/' + PRESIDENT + '_preproc/encdec_' + str(WINDOW) + 'grams_.csv'
df.to_csv(csv_name, index=False)

In [7]:
#assert len(df) == 230020

In [8]:
import torchtext

XFIELD = torchtext.data.Field(sequential=True)
YFIELD = torchtext.data.Field(sequential=True)
DATA = torchtext.data.TabularDataset(csv_name,'csv', 
                                     [('x', XFIELD),('y', YFIELD)], skip_header=True)

XFIELD.build_vocab(DATA)  
YFIELD.build_vocab(DATA)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

In [9]:
from torchtext.data import BucketIterator, Iterator
import torch

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = Iterator(DATA, BATCH_SIZE, device=device, train=True)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [10]:
assert device.type == 'cuda'

# Neural Networks

### Encoder

In [11]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers)

    def forward(self, x, h0, c0):
        x = self.embedding(x).unsqueeze(0)
        out, (h0, c0) = self.lstm(x, (h0, c0))
        return out, (h0, c0)

### Decoder

In [28]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, dropout=0.5)
        self.dense = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim = 1)
  
    def forward(self, x, h0, c0):
        x = self.embedding(x)
        x, (h0, c0) = self.lstm(x, (h0, c0))
        x = self.dense(x.squeeze(0))
        x = self.softmax(x)
        return x, (h0, c0)

### Training

In [29]:
HIDDEN_SIZE = 100
EMBEDDING_SIZE = 100
NUM_LAYERS = 2
ENC_LEARNING_RATE = 0.01
DEC_LEARNING_RATE = 0.01
criterion = nn.NLLLoss()

encoder = Encoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
decoder = Decoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr = ENC_LEARNING_RATE)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr = DEC_LEARNING_RATE)

In [30]:
from tqdm import tqdm

EPOCHS = 10
for ep in range(EPOCHS):
    ep_loss = 0
    
    for batch in tqdm(train_iterator):
        if len(batch) != BATCH_SIZE: break;
        inp = batch.x
        target = batch.y
        
        # init
        loss = 0
        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # encode
        for w in range(inp.size(0)):
            enc_out, (h0, c0) = encoder(inp[w], h0, c0)
            
        # decode
        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        cur = torch.argmax(dec_out,dim=1)
        
        # loss
        target_onehot = torch.nn.functional.one_hot(target.squeeze(), len(YFIELD.vocab))
        loss += criterion(dec_out, target.squeeze())
        
        # optimize
        ep_loss += loss
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        
    print('AVG_LOSS={}, (ABS={})'.format(round((ep_loss/(len(DATA)/BATCH_SIZE)).item(),4), round(ep_loss.item(),2)))

  0%|          | 0/3410 [00:00<?, ?it/s]

AVG_LOSS=6.0101, (ABS=20493.04)


  0%|          | 0/3410 [00:00<?, ?it/s]

AVG_LOSS=5.4543, (ABS=18598.1)


  0%|          | 0/3410 [00:00<?, ?it/s]

...

AVG_LOSS=4.8477, (ABS=16529.61)


  0%|          | 0/3410 [00:00<?, ?it/s]

AVG_LOSS=4.8145, (ABS=16416.31)


100%|█████████▉| 3409/3410 [01:33<00:00, 36.52it/s]

AVG_LOSS=4.7901, (ABS=16333.28)


# Generate Text!

### reload models

In [31]:
pass

### predict

In [34]:
from torch import torch

def voc_index(words):
    return torch.tensor([XFIELD.vocab.stoi[x] for x in words]).to(device)

def predict(inp, RND_FACTOR=0):
    with torch.no_grad():

        h0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        
        for w in range(inp.size(0)):
                enc_out, (h0, c0) = encoder(inp[w], h0, c0)

        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        
        # randomize
        rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR
        cur = torch.argmax(dec_out.add(rnd),dim=1)
        
        # TODO pick more than first one
        return YFIELD.vocab.itos[cur[0].item()]

    
text = ['good', 'evening', 'ladies']
for i in range(100):
    cur_window = text[-WINDOW:]
    vecs = voc_index(cur_window).view(WINDOW,1).repeat(1,BATCH_SIZE)
    text.append(predict(vecs,1.5))
    
text

['good',
 'evening',
 'ladies',
 'and',
 'the',
 'american',
 'people',
 'to',
 'work',
 'to',
 'the',
 'american',
 'people',
 'and',
 'the',
 'united',
 'states',
 'of',
 'america',
 'is',
 'not',
 'a',
 'nation',
 'that',
 '’',
 's',
 'why',
 'we',
 '’',
 're',
 'going',
 'to',
 'make',
 'a',
 'lot',
 'of',
 'america',
 'and',
 'i',
 '’',
 've',
 'got',
 'to',
 'be',
 'a',
 'better',
 'place',
 'we',
 '’',
 've',
 'got',
 'to',
 'be',
 'clear',
 'to',
 'the',
 'world',
 'of',
 'the',
 'world',
 'and',
 'that',
 '’',
 's',
 'why',
 'we',
 '’',
 've',
 'made',
 'a',
 'new',
 'beginning',
 'and',
 'the',
 'world',
 'is',
 'not',
 'just',
 'the',
 'world',
 'is',
 'a',
 'better',
 'and',
 'that',
 '’',
 's',
 'why',
 'we',
 'can',
 'not',
 'be',
 'able',
 'to',
 'do',
 'not',
 'just',
 'the',
 'right',
 'now',
 'and',
 'that',
 '’']